# Setup

Section to set up Jupyter Notebook and intialize experimental settings

### Give Jupyter Notebook access to relative import

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Create GTMDecon object

For ease of user access, we use the GTMDecon python wrapper, built around the gtm-decon C executable files.

In [2]:
from PythonWrapper.GTMDecon import GTMDecon

Initialize GTMDecon wrapper object:

Basic Constructor Arguments:
- **experiment_name** : str [optional]
- **n_topics** : int [optional, default=1]
    - number of topics we wish to set per celltype
- **engine_path** : str
    - path to GTM-decon C executable
    Here we only set the experiment name and engine path, the n_topics parameter will be by default set to 1.

In [3]:
GTM = GTMDecon(
    experiment_name = "gtm-example",
    engine_path = "/home/mcb/users/slaksh1/projects/revision_gb/gtm-decon-phinorm/gtm-decon-plus-noupd-ab-phinorm"
)

We can see the parameters set for our GTM wrapper, including the number of topics per celltype and the engine_path (path to C executable).

We can see that the **experiment_name**, **n_topics**, and **engine_path** attributes have been set as we intended, while the remaining attributes have been left unfilled. The **genes**, **celltypes**, and **bulk_samples** parameters will be populated as we provide our input reference and bulk data.

In [4]:
print(GTM)

GTM-decon wrapper object with attributes:
  - experiment_name: gtm-example
  - n_topics: 1
  - engine_path: /home/mcb/users/slaksh1/projects/revision_gb/gtm-decon-phinorm/gtm-decon-plus-noupd-ab-phinorm
  - genes: []
  - celltypes: []
  - bulk_samples: []
  - verbose: True
  - output_intermediates: False
  - override_geneset: False



# Example Deconvolution Pipeline

In order to infer cell-type proportions for a given bulk dataset and given single cell reference matrix, we can use the **GTMDecon.pipeline** function to process the input information, run it through the gtm-decon C executables, and output the predicted cell-type proportions of our bulk.

### Loading DataFrames

In [5]:
import pandas as pd
import anndata as ad

Load our reference and bulk dataframes from the example csvs.

The **reference_DataFrame** should be a pandas DataFrame object, the rows are cells, the columns are the genes, with one additional column named *Celltype* containing the cell-type labels associated with each row.

The **bulk_DataFrame** should be a pandas DataFrame, where the rows represent genes, with the genes stored as the index, and the columns represent the bulk batches.

In [6]:
bulk_DataFrame = pd.read_csv("../data/bulk.csv", index_col=0)
reference_DataFrame = pd.read_csv("../data/ref.csv")

### Running our Pipeline

GTMDecon.pipeline arguments:
- **bulk_data** : pd.DataFrame
- **reference_data** : pd.DataFrame
- **directory** : str
    - directory where we want to save the model parameters and inferred cell-type proportions 
    - we expect the inferred propotions to end up here: **/vignette_results/gatheredResults.csv**


We make a directory to store the results for this vignette

In [7]:
!mkdir vignette_results

mkdir: cannot create directory ‘vignette_results’: File exists


Here we run our pipeline, including processing data to GTM-decon format, training, and cell-type proportion inference.

In [8]:
GTM.pipeline(
    bulk_data = bulk_DataFrame,
    reference_data = reference_DataFrame,
    directory = os.path.join(os.getcwd(), 'vignette_results'),
)

Running GTM Deconvolution Pipeline
Writing results to /home/mcb/users/zhuang35/projects/gtm-decon/vignettes/vignette_results
**********************************

Saving genes file to /home/mcb/users/zhuang35/projects/gtm-decon/vignettes/vignette_resultsgenes.txt ...
Successfully wrote genes file to /home/mcb/users/zhuang35/projects/gtm-decon/vignettes/vignette_resultsgenes.txt
Saving genes file to /home/mcb/users/zhuang35/projects/gtm-decon/vignettes/vignette_resultsgenes.txt ...
Successfully wrote meta file to /home/mcb/users/zhuang35/projects/gtm-decon/vignettes/vignette_resultsmeta.txt
Saving training file to /home/mcb/users/zhuang35/projects/gtm-decon/vignettes/vignette_resultstrainData.txt ...
Successfully wrote training file to /home/mcb/users/zhuang35/projects/gtm-decon/vignettes/vignette_resultstrainData.txt
Saving prior file to /home/mcb/users/zhuang35/projects/gtm-decon/vignettes/vignette_resultspriorData.txt ...
Successfully wrote prior file to /home/mcb/users/zhuang35/projec

In [12]:
predicted_props = pd.read_csv("/home/mcb/users/zhuang35/projects/gtm-decon/vignettes/vignette_results/gatheredResults.csv", index_col=0)

In [13]:
predicted_props.head()

,MHC class II cell,PSC cell,acinar cell,alpha cell,beta cell,co-expression cell,delta cell,ductal cell,endothelial cell,epsilon cell,gamma cell,mast cell
H1,0.085408,0.008100,0.135150,0.095004,0.178512,0.011165,0.110165,0.026610,0.015685,0.082392,0.104152,0.147656
D1,0.078927,0.051307,0.023607,0.186299,0.198460,0.019311,0.104674,0.066449,0.039126,0.019692,0.042736,0.169410
